<a href="https://colab.research.google.com/github/gargarchit/Federated-Learning-with-Encrypted-Gradient-Aggregation/blob/master/Federated_Learning_with_Encrypted_Gradient_Aggregation_using_PyTorch_%2B_PySyft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Need to train on the MNIST dataset using federated learning However the gradient should not come up to central server in raw form**

In [0]:
import torch
from torchvision import datasets, transforms

<h2>Installing and Importing PySyft

In [0]:
!pip install tf-encrypted

! URL="https://github.com/openmined/PySyft.git" && FOLDER="PySyft" && if [ ! -d $FOLDER ]; then git clone -b dev --single-branch $URL; else (cd $FOLDER && git pull $URL && cd ..); fi;

!cd PySyft; python setup.py install  > /dev/null

import os
import sys
module_path = os.path.abspath(os.path.join('./PySyft'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
!pip install --upgrade --force-reinstall lz4
!pip install --upgrade --force-reinstall websocket
!pip install --upgrade --force-reinstall websockets
!pip install --upgrade --force-reinstall zstd

In [0]:
import syft as sy
hook = sy.TorchHook(torch)

# Create a couple of workers
bob = sy.VirtualWorker(hook, id="bob")  
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")
# clearing data of virutal workers.
bob.clear_objects
alice.clear_objects
secure_worker.clear_objects
compute_nodes = [bob, alice]

In [16]:
compute_nodes

[<VirtualWorker id:bob #objects:72>, <VirtualWorker id:alice #objects:60>]

In [38]:
#We define the setting of the learning task

class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 10000
        self.epochs = 50
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

torch.manual_seed(args.seed)

In [39]:
use_cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


<h2>Data loading and sending to workers</h2>

In [0]:
# FederatedDataLoader 
# we distribute the dataset across all the workers, it's now a FederatedDataset
transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))]) 
mnist_trainset = datasets.MNIST(root='../data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=args.test_batch_size, shuffle=True)


mnist_testset = datasets.MNIST(root='../data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=args.test_batch_size, shuffle=True)


In [0]:
train_distributed_dataset = []

for batch_idx, (data,target) in enumerate(train_loader):
    data = data.send(compute_nodes[batch_idx % len(compute_nodes)])
    target = target.send(compute_nodes[batch_idx % len(compute_nodes)])
    train_distributed_dataset.append((data, target))

In [0]:
#train_distributed_dataset

<h2>Network Architecture</h2>

In [0]:
from torch import nn, optim
import torch.nn.functional as F
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [0]:
class Classifier(nn.Module):
    """
    Forward Convolutional Neural Network Architecture model
    
    """
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [0]:
model = Classifier()
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr)

In [0]:
for epoch in range(1, args.epochs + 1):
    model.train()
    for batch_idx, (data, target) in enumerate(train_distributed_dataset): # iterate through each worker's dataset
        
        model.send(data.location) #send the model to the right location
        
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad() # 1) erase previous gradients (if they exist)
        output = model(data)  # 2) make a prediction
        loss = F.nll_loss(output, target)  # 3) calculate how much we missed
        loss.backward()  # 4) figure out which weights caused us to miss
        optimizer.step()  # 5) change those weights
        model.get()  # get the model back (with gradients)
        
        if batch_idx % args.log_interval == 0:
            loss = loss.get() #get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * data.shape[0], len(train_loader),
                       100. * batch_idx / len(train_loader), loss.item()))